In [83]:
#Importar bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns


In [84]:
#Importar tabela
file_path = 'C:/Users/Gabriel/Documents/Gabriel/py/dataset-musica-saude.csv'
df = pd.read_csv(file_path, sep=';')
df.head(10)

,Timestamp,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Music effects,Permissions
0,8/27/2022 19:29:02,18.0,Spotify,3.0,Yes,Yes,Yes,Latin,Yes,Yes,...,Sometimes,Very frequently,Never,Sometimes,3.0,0.0,1.0,0.0,NaN,I understand.
1,8/27/2022 19:57:31,63.0,Pandora,1.5,Yes,No,No,Rock,Yes,No,...,Sometimes,Rarely,Very frequently,Rarely,7.0,2.0,2.0,1.0,NaN,I understand.
2,8/27/2022 21:28:18,18.0,Spotify,4.0,No,No,No,Video game music,No,Yes,...,Never,Rarely,Rarely,Very frequently,7.0,7.0,10.0,2.0,No effect,I understand.
3,8/27/2022 21:40:40,61.0,YouTube Music,2.5,Yes,No,Yes,Jazz,Yes,Yes,...,Sometimes,Never,Never,Never,9.0,7.0,3.0,3.0,Improve,I understand.
4,8/27/2022 21:54:47,18.0,Spotify,4.0,Yes,No,No,R&B,Yes,No,...,Very frequently,Very frequently,Never,Rarely,7.0,2.0,5.0,9.0,Improve,I understand.
5,8/27/2022 21:56:50,18.0,Spotify,5.0,Yes,Yes,Yes,Jazz,Yes,Yes,...,Very frequently,Very frequently,Very frequently,Never,8.0,8.0,7.0,7.0,Improve,I understand.
6,8/27/2022 22:00:29,18.0,YouTube Music,3.0,Yes,Yes,No,Video game music,Yes,Yes,...,Rarely,Never,Never,Sometimes,4.0,8.0,6.0,0.0,Improve,I understand.
7,8/27/2022 22:18:59,21.0,Spotify,1.0,Yes,No,No,K pop,Yes,Yes,...,Sometimes,Rarely,Never,Rarely,5.0,3.0,5.0,3.0,Improve,I understand.
8,8/27/2022 22:33:05,19.0,Spotify,6.0,Yes,No,No,Rock,No,No,...,Never,Never,Very frequently,Never,2.0,0.0,0.0,0.0,Improve,I understand.
9,8/27/2022 22:44:03,18.0,I do not use a streaming service.,1.0,Yes,No,No,R&B,Yes,Yes,...,Sometimes,Rarely,Sometimes,Sometimes,2.0,2.0,5.0,1.0,Improve,I understand.


In [85]:
#Pré-processamento

# Remover a coluna 'permissions'
df = df.drop(columns=['Permissions', 'Timestamp', 'Primary streaming service'])

# Atribuir um peso a cada resposta nas colunas de frequencia
peso_respostas = {
    'Never': 0,
    'Rarely': 1,
    'Sometimes': 2,
    'Very frequently': 3
}

# Coluna inicial e final das frequencias para aplicar o peso de cada frequencia
coluna_inicial = 'Frequency [Classical]'
coluna_final = 'Frequency [Video game music]'

# Selecionar todas as colunas entre 'coluna_inicial' e 'coluna_final'
colunas_musicas = df.loc[:, coluna_inicial:coluna_final].columns

# Troca dos valores antigos para o peso
df[colunas_musicas] = df[colunas_musicas].replace(peso_respostas)

# Mapear 'yes' para 1 e 'no' para 0
boolean_mapping = {
    'Yes': 1,
    'No': 0
}

# Lista das colunas para transformar em booleano
colunas_booleanas = ['While working', 'Instrumentalist', 'Composer', 'Exploratory', 'Foreign languages']

# Aplica o mapping
df[colunas_booleanas] = df[colunas_booleanas].replace(boolean_mapping)

# Substituir 'Improve' por 1, 'No effect' e valores vazios por 0, e remover o restante
df['Music effects'] = df['Music effects'].apply(lambda x: 1 if x == 'Improve' else (0 if x == 'No effect' or pd.isnull(x) else None))

# Remover as linhas onde o valor em 'Music effects' é None (ou seja, não é 'Improve' nem 'No effect')
df.dropna(subset=['Music effects'], inplace=True)

# Formatar colunas_booleanas para boolean
df[colunas_booleanas] = df[colunas_booleanas].astype(bool)



C:\Users\Gabriel\AppData\Local\Temp\ipykernel_3888\3038362490.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[colunas_musicas] = df[colunas_musicas].replace(peso_respostas)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_3888\3038362490.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[colunas_booleanas] = df[colunas_booleanas].replace(boolean_mapping)


In [86]:
from sklearn.preprocessing import LabelEncoder

# Aplicando o LabelEncoder na coluna Fav Genre
label_encoder = LabelEncoder()
df['Fav_Genre_Encoded'] = label_encoder.fit_transform(df['Fav genre'])

df = df.drop(columns='Fav genre')





In [87]:
#Selecionar colunas para o eixo X(features)
# colunas_perfil = ['Age', 'Hours per day', 'While working','Instrumentalist', 'Composer', 'Exploratory', 'Foreign languages', 'Fav_Genre_Encoded']
# colunas_transtorno = ['Anxiety', 'Depression', 'Insomnia', 'OCD']
# colunas_x = colunas_perfil + colunas_transtorno + list(colunas_musicas)

# Trate os NaN (escolha uma das opções)
# df[colunas_x] = df[colunas_x].fillna(df[colunas_x].mean())

# Trata os NaN somente nas colunas numericas
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype in ['float64', 'int64'] else x)


In [88]:
#Desenvolvimento algoritmo

import matplotlib.pyplot as mtp  

target_column = 'Music effects'

x = df.drop(columns=[target_column, ]).values
y = df[target_column].values

# Dividir o dataset entre treino e teste
from sklearn.model_selection import train_test_split
  
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 42)  

#Padronizar o X
from sklearn.preprocessing import StandardScaler  
sc = StandardScaler()  
x_train = sc.fit_transform(x_train)  
x_test = sc.transform(x_test)  

In [89]:
# Criando set de treino no Naive Bayes
from sklearn.naive_bayes import GaussianNB  
classifier =  GaussianNB()
classifier.fit(x_train, y_train) 

# Prevendo o set de resultados
y_pred = classifier.predict(x_test)  

In [90]:
#Relátorios

# Verificar precisão
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Confusion Matrix
from sklearn.metrics import confusion_matrix  
cm = confusion_matrix(y_test, y_pred)  
print(cm)

# Converter o relatório em um DataFrame
from sklearn.metrics import classification_report 

# Criar o relatório de classificação
report = classification_report(y_test, y_pred, output_dict=True)  # output_dict=True retorna um dicionário

report_df = pd.DataFrame(report).transpose() 
report_df = report_df[['precision', 'recall', 'f1-score', 'support']] 

# Exibir a tabela
print(report_df)


Accuracy: 0.6481
[[ 9 16]
 [22 61]]
              precision    recall  f1-score     support
0.0            0.290323  0.360000  0.321429   25.000000
1.0            0.792208  0.734940  0.762500   83.000000
accuracy       0.648148  0.648148  0.648148    0.648148
macro avg      0.541265  0.547470  0.541964  108.000000
weighted avg   0.676031  0.648148  0.660400  108.000000
